# Option Chain Data

In [42]:
import pandas as pd
from breeze_connect import BreezeConnect
from datetime import date
from selenium import webdriver
from pyotp import TOTP
import urllib
import time
import warnings
warnings.filterwarnings('ignore')

Login

In [43]:
file = open("security.txt", "r")
keys = file.read().split() 
api_key = keys[0]
key_secret = keys[1]
userID = keys[2]
pwd = keys[3]
totp_key = keys[4]
breeze = BreezeConnect(api_key=api_key)

In [44]:
browser = webdriver.Chrome()
browser.get("https://api.icicidirect.com/apiuser/login?api_key="+urllib.parse.quote_plus(api_key))
browser.implicitly_wait(5)
username = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[1]/input')
password = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[3]/div/input') 
username.send_keys(userID)
password.send_keys(pwd)
#Checkbox
browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[4]/div/input').click()
# Click Login Button
browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[5]/input[1]').click()
time.sleep(2)
pin = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[2]/div/div[2]/div[2]/div[3]/div/div[1]/input')
totp = TOTP(totp_key)
token = totp.now()
pin.send_keys(token)
browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[2]/div/div[2]/div[2]/div[4]/input[1]').click()
time.sleep(3)
temp_token=browser.current_url.split('apisession=')[1][:8]
print('temp_token', temp_token)
breeze.generate_session(api_secret=key_secret,session_token=temp_token)

temp_token 6293895


Function to fetch option chain

In [45]:
def Get_option_chain(stock_code, expiry_date, right):
    try:
        data = breeze.get_option_chain_quotes(stock_code=stock_code,
                        exchange_code="NFO",
                        product_type="options",
                        expiry_date=expiry_date,
                        right=right,
                        )
        put_data = pd.DataFrame(data["Success"])
        if put_data.empty:
            print('Data not found:')
            df = pd.DataFrame()
            return df
        else:
            df = put_data
            return df
    except Exception as e:
        print("Get_option_chain failed: {}".format(e))

Get Nearest Expiry from Instruments file

In [46]:
global scrip
scrip = pd.read_csv("Data/FONSEScripMaster.txt")
scrip = scrip[['Token', 'InstrumentName', 'ShortName', 'Series','ExpiryDate', 'StrikePrice', 'OptionType', 'ExchangeCode']]


In [47]:
def get_nearest_expiry(index_name):
    global scrip
    scrip = scrip[scrip.ShortName == index_name]
    scrip = scrip[scrip.Series == 'OPTION']
    scrip.ExpiryDate = pd.to_datetime(scrip.ExpiryDate)
    scrip['ExpiryDate'] = [x.date() for x in scrip.ExpiryDate]
    return sorted(scrip.ExpiryDate)[0]

Fetch Option Chain for Call

In [48]:
stock_code = 'CNXBAN'
icici_code = breeze.get_names(exchange_code = 'NSE',stock_code = stock_code).get('isec_stock_code')
expiry_date = get_nearest_expiry(icici_code) 
expiry_date_iso = expiry_date.isoformat()[:10] + 'T05:30:00.000Z'
op_chain_call = Get_option_chain(icici_code, expiry_date_iso, 'call')
op_chain_call.to_csv('Data/op_chain_call.csv')
print('Done')

Done


Filter Important Columns

In [49]:
op_chain_call = op_chain_call[['expiry_date', 'right',  'strike_price', 'ltp', 'open_interest', 'chnge_oi', 'total_buy_qty', 'total_sell_qty']]
op_chain_call.sample(10)

,expiry_date,right,strike_price,ltp,open_interest,chnge_oi,total_buy_qty,total_sell_qty
124,23-Mar-2023,Call,48500.0,0.00,0.0,0.0,0,0
56,23-Mar-2023,Call,39400.0,540.00,519525.0,247525.0,38775,18000
42,23-Mar-2023,Call,38000.0,1754.00,59175.0,5300.0,82325,4375
126,23-Mar-2023,Call,49500.0,0.00,0.0,0.0,0,0
121,23-Mar-2023,Call,47000.0,0.00,0.0,0.0,0,0
99,23-Mar-2023,Call,43700.0,0.00,0.0,0.0,0,0
33,23-Mar-2023,Call,37100.0,2588.95,300.0,200.0,4700,4675
114,23-Mar-2023,Call,45200.0,0.00,0.0,0.0,0,0
7,23-Mar-2023,Call,32500.0,0.00,0.0,0.0,0,0
64,23-Mar-2023,Call,40200.0,154.90,1088400.0,526875.0,154475,32250


Filter Important Calls near LTP

In [50]:
op_chain_call = op_chain_call[(op_chain_call.strike_price > 40000) & (op_chain_call.strike_price < 42000)]
op_chain_call

,expiry_date,right,strike_price,ltp,open_interest,chnge_oi,total_buy_qty,total_sell_qty
63,23-Mar-2023,Call,40100.0,185.85,448800.0,210650.0,136125,33525
64,23-Mar-2023,Call,40200.0,154.90,1088400.0,526875.0,154475,32250
65,23-Mar-2023,Call,40300.0,122.75,695100.0,231075.0,149725,44825
66,23-Mar-2023,Call,40400.0,100.60,549675.0,270550.0,133425,28525
67,23-Mar-2023,Call,40500.0,82.00,1566125.0,1300.0,235500,103000
68,23-Mar-2023,Call,40600.0,67.00,388275.0,96700.0,154050,36275
69,23-Mar-2023,Call,40700.0,53.50,370350.0,-51825.0,140275,33550
70,23-Mar-2023,Call,40800.0,41.50,595025.0,208025.0,131600,37125
71,23-Mar-2023,Call,40900.0,33.95,319800.0,-7725.0,126825,19875
72,23-Mar-2023,Call,41000.0,27.60,2061975.0,385975.0,184800,117875


Filter Strike Price of Call option on the basis of premium

In [51]:
premium_call = 100
op_chain_call = op_chain_call[op_chain_call.ltp > 0]
op_chain_call.premium = premium_call
op_chain_call['diff'] = (op_chain_call.ltp - op_chain_call.premium).abs()
op_chain_call = op_chain_call.sort_values(by=['diff'])
strike_price_call = op_chain_call.strike_price.iloc[0]
print(f"{strike_price_call}")

40400.0
